In [0]:
from pyspark.sql.functions import col, avg, window

# 1. Read from the Silver table
df_silver_source = spark.readStream.table("weather_silver_cleaned")

# 2. Aggregate: Average Temp & Humidity using the CORRECT column name (event_hub_time)
df_gold = (df_silver_source
    .withWatermark("event_hub_time", "10 minutes") # Wait for late data
    .groupBy(
        window(col("event_hub_time"), "1 hour"),     # Create 1-hour "buckets"
        col("city")
    )
    .agg(
        avg("temp_celsius").alias("avg_temp"),
        avg("humidity_pct").alias("avg_humidity")
    )
    .select(
        col("window.start").alias("start_time"),
        col("window.end").alias("end_time"),
        col("city"),
        col("avg_temp"),
        col("avg_humidity")
    )
)

# 3. Write to Gold Table
(df_gold.writeStream
    .format("delta")
    .outputMode("complete") # 'complete' is required for streaming aggregations
    .option("checkpointLocation", "/mnt/weather/checkpoints/gold")
    .toTable("weather_gold_summary"))

In [0]:
display(spark.read.table("weather_gold_summary"))

In [0]:
from pyspark.sql.functions import col, avg, window

# 1. Read from the Silver table
df_silver_source = spark.readStream.table("weather_silver_cleaned")

# 2. Aggregate: Average Temp & Humidity every 1 hour
df_gold = (df_silver_source
    .withWatermark("event_hub_time", "10 minutes") # Handle late data
    .groupBy(
        window(col("event_hub_time"), "1 hour"),
        col("city")
    )
    .agg(
        avg("temp_celsius").alias("avg_temp"),
        avg("humidity_pct").alias("avg_humidity")
    )
    .select(
        col("window.start").alias("start_time"),
        col("window.end").alias("end_time"),
        col("city"),
        col("avg_temp"),
        col("avg_humidity")
    )
)

# 3. Write to Gold Table
(df_gold.writeStream
    .format("delta")
    .outputMode("complete") # 'Complete' is required for aggregations
    .option("checkpointLocation", "/mnt/weather/checkpoints/gold")
    .toTable("weather_gold_summary"))